In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

__Cascade Classifiers 로드__

In [3]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
eyesplit_cascade = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

__advanced_eye_detect 함수 정의__

In [4]:
def advanced_eye_detect(img,cascade,info):
    
    ret_val=[]
    
    img_gray =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(img_gray)
    
    if len(faces)!=1:
        return ret_val
    
    for (x,y,w,h) in faces:
        roi_gray = img_gray[y:y+h, x:x+w]
        eyes =cascade.detectMultiScale(roi_gray,1.1,4)
    
    if not (len(eyes)==2):    
        if info == 'normal':
            return advanced_eye_detect(img,eyesplit_cascade,'split')
        elif info == 'split':
            return advanced_eye_detect(img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
        
    ret_val = eyes
    return ret_val

__get_rotated_image 함수 정의__

In [5]:
def get_rotated_image(img,eyes):
    
    eye_1 , eye_2 = eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
        
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center

    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    if not delta_x or delta_y:
        return img
    
    angle=np.arctan(delta_y/delta_x)
    angle = (angle * 180) / np.pi
    
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, (angle), 1.0)
    rotated_img = cv2.warpAffine(img, M, (w, h))
  
    return rotated_img

__calculate_rotated_eyes 함수 정의__

In [6]:
def calculate_rotated_eyes(rotated_img,cascade,info):

    ret_val=[]
    
    rotated_gray = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(rotated_gray, 1.1, 4)
    
    if len(faces)!=1:
        return ret_val
    
    for (x,y,w,h) in faces:

        black=np.zeros(rotated_gray.shape,dtype='uint8')
        black[y:y+h, x:x+w]=rotated_gray[y:y+h, x:x+w]

        rotated_eyes =cascade.detectMultiScale(black,1.1,4)    
    
    if len(rotated_eyes)!=2:    
        if info == 'normal':
            return calculate_rotated_eyes(rotated_img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
    
    ret_val = rotated_eyes
    
    return ret_val

__extract_facial_mask_area 함수 정의__

In [7]:
def extract_facial_mask_area(rotated_img,rotated_eyes):
    
    eye_1 , eye_2 = rotated_eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
    
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center
    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    L = np.sqrt(delta_x**2 + delta_y**2)
    xpad_L , xpad_R = int(0.6*L) , int(1.6*L)
    ypad_U , ypad_D = int(0.6*L) , int(1.8*L)
    
    ROI = rotated_img[left_eye_y-ypad_U:left_eye_y+ypad_D,left_eye_x-xpad_L:left_eye_x+xpad_R]
    ROI_resized = cv2.resize(ROI,(120,140))
    mask_area = ROI_resized[50:140,0:120]
    
    return mask_area

__메인 코드__

In [16]:
import os

img_path = "C:/img_sample/nose_mask_2000/" # replace it with your path

files = os.listdir(img_path)
image_files = [file for file in files if file.endswith(".png")]
mask_area=[]
for file in image_files:
    
    image=cv2.imread(img_path+file)
    print(f'{file} detection result')
    
    if image.shape[0]<400 or image.shape[1]<400 :
        w,h,c = image.shape
        image = cv2.resize(image,(h*2,w*2))
    
    eyes = advanced_eye_detect(image,eye_cascade,'normal')
    
    if len(eyes)!=2:
        print('Initial Detection Failed : not human or invalid image')
        print('='*65)
        continue
    
    print('Initial Detection success')
    rotated_image = get_rotated_image(image,eyes)
    rotated_eyes = calculate_rotated_eyes(rotated_image,eye_cascade,'normal')
    
    if len(rotated_eyes)!=2:
        print('Rotated Detection Failed : not human or invalid image')
        print('='*65)
        continue
        
    print('Rotated Detection success')
    print('='*65)
    
    current_mask_area = extract_facial_mask_area(rotated_image,rotated_eyes)
    mask_area.append(current_mask_area)
    cv2.imwrite('C:/img_sample/nose_mask_2000_extracted/'+file, current_mask_area) # replace it with your path

09292-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09293-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09294-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09295-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09296-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09297-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09298-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09299-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09300-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09301-with-nose-mask(nose_tip).png detection result
Initial 

Initial Detection success
Rotated Detection success
09343-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09344-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09345-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09346-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09347-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09348-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09349-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09350-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09351-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid imag

Initial Detection success
Rotated Detection success
09394-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09395-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09396-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09397-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09398-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09399-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09400-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09401-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09402-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
0940

Initial Detection Failed : not human or invalid image
09445-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09446-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09447-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09448-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09449-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09450-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09451-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection Failed : not human or invalid image
09452-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09453-with-nose-mask(nose_tip).png detection result
Initial Detection success


Initial Detection Failed : not human or invalid image
09496-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09497-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09498-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09499-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09500-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09501-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09502-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09503-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
09504-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
09

...

Initial Detection success
Rotated Detection success
11063-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11064-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11065-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11066-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11067-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11068-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11069-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11070-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11071-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid imag

Initial Detection Failed : not human or invalid image
11112-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11113-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11114-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11115-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11116-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11117-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11118-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11119-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11120-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11

Rotated Detection success
11162-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11163-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11164-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection Failed : not human or invalid image
11165-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11166-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11167-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11168-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11169-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11170-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11

Initial Detection success
Rotated Detection success
11211-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11212-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11213-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11214-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11215-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11216-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11217-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11218-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11219-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
1122

Initial Detection Failed : not human or invalid image
11265-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11266-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11267-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11268-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11269-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11270-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11271-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
11272-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
11273-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid im

In [17]:
print(len(mask_area))

790


In [3]:
import os


['chin_mask', 'full_mask', 'nose_mask', 'no_mask']

In [17]:
import os
import numpy as np
np.random.seed(1120)
from sklearn.model_selection import train_test_split
import cv2

top_path = 'C:/img_sample/temp/' # 안에 chin_mask , full_mask... 로 하위 폴더가 있는 상위 폴더 경로

save_path = 'C:/mask_data/' # 저장할 경로 (안에 빈 train , test 폴더 먼저 만들어 놓고 실행)

folders = os.listdir(top_path)

for folder in folders:
    
    files = os.listdir(top_path+folder+'/')
    
    image_files = [file for file in files if file.endswith(".png")]
    
    image_1000 = image_files[-1000:]
    
    train,test = train_test_split(image_1000,test_size=0.3)
    
    for img in train:
        original=cv2.imread(top_path+folder+'/'+img)
        cv2.imwrite(save_path+'train/'+folder+'/'+img, original.copy())
        
    for img in test:
        original=cv2.imread(top_path+folder+'/'+img)
        cv2.imwrite(save_path+'test/'+folder+'/'+img, original.copy())

In [14]:
folder

'chin_mask'

In [6]:
a=list(range(50))
b,c = train_test_split(a,test_size=0.3)

In [8]:
len(b)

35

In [9]:
len(c)

15